In [ ]:
import sys
import os
import time
# import visa
import logging
import pyvisa as visa
import numpy             as np

import chipwhisperer as cw

#----------- libraries for serial(Dut Pico board) ---------
import serial.tools.list_ports
from serial import Serial
from serial.tools import list_ports

#-------------Oscilloscope object --------------------           

# scope functionality
class Scope(object):

  def __init__(self):
    self.open()
    self.valid_trigger_states = ['AUTO', 'NORM', 'SINGLE', 'STOP']

  def __del__(self):
    self.close()

  def open(self):
   #  get all resources connected to PC
    self.rm    = visa.ResourceManager()
   #  open vcip protocol
    try:
      self.scope = self.rm.open_resource('TCPIP0::192.168.1.79::inst0::INSTR') #insert your scope IP here
      self.reset()
      self.scope.timeout=5000
      self.scope.clear()
      r=self.scope.query(r"""vbs? 'return=app.WaitUntilIdle(5)' """)
    except:
      self.scope = None
      logging.info("Unable to locate the scope VISA interface")

  def close(self):
    # disconnect the oscilloscope
    # there is a bug in the pyvisa impl. so we'll wrap in
    # a try/catch
    try:
      if (self.scope is not None):
        self.scope.close()

      if (self.rm is not None):
        self.rm.close()

    except:
      logging.info("pyvisa error in closing resource")
  # Set up for a certain trace acquisition process --- Si 2017.12.5 Added
  # vdiv:         vertical resolution (V/div)
  # timebase:     horizontal resolution (S/div)
  # samplerate:   Fixed sample rate (Sa/s)
  # duration:     Measure Duration (usually set as 10*timebase, might not work otherwise)
  # voffset:      Vertical offset
  def setup(self,vdiv,timebase,samplerate,duration,voffset):
    # setup for the trace acquisition
    # set time base
    if (self.scope):
      self.scope.write("C2:TRA ON")
    if(self.scope):
      self.scope.write(r"""vbs 'app.Acquisition.ClearSweeps' """)# clear
    if (self.scope):
      self.scope.write("TDIV " + timebase)
    if (self.scope):
      self.scope.write("C2:VDIV " + vdiv)
    # set waveform format
    if (self.scope):
      self.scope.write("CFMT DEF9,WORD,BIN")
    # set Sampling Rate
    if (self.scope):
      self.scope.write(r"""vbs 'app.Acquisition.Horizontal.Maximize = "FixedSampleRate" '""")
      self.scope.write(r"""vbs 'app.Acquisition.Horizontal.SampleRate = "%s" '""" %samplerate)
      self.scope.write(r"""vbs 'app.Acquisition.Horizontal.AcquisitionDuration = "%s" '"""%duration)
      self.scope.write(r"""vbs 'app.Acquisition.C2.VerOffset = "%s" '""" % voffset)
  # Set up trigger condition
  # delay: negative means start acuisition at post trigger xx s; positive shows the percent of trace before trigger
  # level: the level of trigger which counts as one valid trace
  def set_trigger(self,delay,level):
    # set trigger mod to single
    if (None == self.scope):
      self.open()
    # set trigger delay9
    
    if (self.scope):
      self.scope.write("C1:TRA ON")
    if (self.scope):
      self.scope.write("C1:TRCP DC")
    if (self.scope):
      self.scope.write("TRDL " + delay)
    # set trigger level
    if (self.scope):
      self.scope.write("C1:TRLV " + level)
    # set triggr positive edge
    if (self.scope):
      self.scope.write("C1:TRSL POS")
  # Stop the previous capture and start a new one
  def start_trigger(self):
    # set trigger state
    if (self.scope):
      self.scope.write("TRMD SINGLE") # Used for single waveform capture. To save querying time
      #stop
      self.scope.write(r"""vbs 'app.acquisition.triggermode = "stopped" ' """)
      r = self.scope.query(r"""vbs? 'return=app.WaitUntilIdle(.01)' """)
      self.scope.write(r"""vbs 'app.acquisition.triggermode = "single" ' """)
      r = self.scope.query(r"""vbs? 'return=app.WaitUntilIdle(.01)' """)
  # get the triggering status
  def get_trigger(self):
    # read trigger state from the oscilloscope
    if (None == self.scope):
      self.open()

    if (self.scope):
      ret = self.scope.query("TRMD?")
      return ret.split()[1]

  # check whether Lecroy has already been triggered
  # This is poor-mans timeout. There
  # doesn't seem to be a good platform independant
  # way to raise a timeout exception so this will
  # have to do
  # wait untill the trigger is activated and the capture stops
  def wait_for_trigger(self):
    if (None == self.scope):
      self.open()

    if (self.scope):
        for tries in range(10):
            if (self.get_trigger() == 'STOP'):
                return True
            else:
                time.sleep(0.5)

    print("Trigger timout")
    return False
  # Read one trace back
  # samples: number of samples on that trace
  # isshort: short or float
    

  def get_channel(self,samples,isshort,channel='C2'):
    # read channel data
    if (None == self.scope):
        self.open()
    if (self.scope):
        if(isshort):
            cmd = self.scope.write('{0}:WF? DATA1'.format(channel))
            trc = self.scope.read_raw()
            #print(trc)
            hsh = trc.find(b'#', 0)
            skp = int(trc[hsh + 1:hsh + 2])
            trc = trc[hsh +skp+2:-1]
            #et = np.fromstring(trc, dtype='<h', count=samples)
            #ret = np.fromstring(trc, dtype='<h', count=samples)
            ret = np.frombuffer(trc, dtype='<h', count=samples )
            return ret
        else:
            cmd = self.scope.write('{0}:INSPECT? "SIMPLE"'.format(channel))
            trc = self.scope.read_raw()
           # print(trc)
            hsh = trc.find(b'\n', 0)
            trc = trc[hsh+2:-1]
            ret = np.fromstring(trc, dtype='float', sep=' ', count=samples)
            #ret = np.frombuffer(trc, dtype='float', sep=' ', count=samples)
        return ret
    else:
        return None

  def reset(self):# reset the ocssiloscope
    if (self.scope is not None):

        logging.info("resetting oscilloscope!")
#         self.scope.write("*RCL 6") # Reseting the set panel to 6.

        time.sleep(1)
        not_ready = True

        while(not_ready):
            ret = self.scope.query("INR?")
            ret = ret.split()[1]
            not_ready = (0x01 == ((int(ret) >> 13) & 0x01))
            time.sleep(0.5)

    return

#Scope setup function to initilise Lecroy. Please setup the IP address in the Scope.open() method above 
# look for this statement: self.scope = self.rm.open_resource('TCPIP0::192.168.1.79::inst0::INSTR')

def scope_setup(num_of_samples=120000,sample_rate=50E6,isshort='False',vdiv=2.5E-3,trg_delay="0",trg_level="1.65V",voffset='0'):

    # Acquisition setup: find this setting through the capturing traces from the setup program
    #num_of_traces=10000                                          # number of traces
    # num_of_samples=600                                      # number of samples
    # sample_rate=10E9                                           # Sample Rate
    # isshort=False                                             # Type of the samples: short or float
    # vdiv=2.5E-3 #1.8E-2                                                 # Vertical resolution: V/div
    # trg_delay ="0"# "-311.2US"                                      # Trigger delay: negative means post trigger (S)
    # trg_level = "1.65V"                                            # Trigger level: start capture when trigger passes it
    # #isenc = True                                                # Perform encryption/decryption
    # voffset="0"#"-11.4 mV"                                          # Vertical Offset

    # In most common cases, you do not have to change anything below this line
    # Compute the setup parameters from above
    xscale=1/sample_rate                                        # sampling interval (s)
    duration=xscale*num_of_samples                              # sample duration (s)
    # For short type of acquisition, the captured trace is scaled in order to store in a 16 bit integer
    # yscale saves the scale value, in case you need to reconstruct the real traces
    # For float type of acquisition, the samples are exactly the real samples on the trace, so yscale=1
    if(isshort):
        yscale=vdiv/(65536/10.0)
        print("isshort")
    else:
        yscale=1
        print("False")
    timebase=str(xscale*num_of_samples/10)+"S"                  # timebase: s/div


    #Open scope
    oscope = Scope()
    # setup the scope
    oscope.setup(str(vdiv)+"V",timebase,str(sample_rate/1E6)+"MS/s",str(duration)+"S",voffset)
    # set trigger
    oscope.set_trigger(trg_delay,trg_level)
    return oscope

#DUT setup for communication start and bit stream program for CW305 and Picochip 
def dut_setup(board = "pico", key = [0], bitfile = None):
    if(board == "CW305"):
        target = cw.target(None, cw.targets.CW305,fpga_id = '35t',force =True, bsfile=bitfile)  

        #Configuration of the PLL Clocks
        target.pll.pll_enable_set(True) #Enable PLL chip
        target.pll.pll_outenable_set(False, 0) # Disable unused PLL0
        target.pll.pll_outenable_set(True, 1)  # Enable PLL
        target.pll.pll_outenable_set(False, 2) # Disable unused PLL2
        # set PLL 0 to 10MHz
        target.pll.pll_outfreq_set(100E6, 1)
          
        return target
    elif(board == "pico"): 
          #-------------- Serial object -----------------
        port_list = list(serial.tools.list_ports.comports())
        print(port_list)
        if len(port_list) == 0:
            print('No Available Serial Port')
        else:
            for i in range(0,len(port_list)):
                print(port_list[i])

        #----------------------------------------------
        comports = []
        for x in list_ports.comports():
            cp = Serial()
            cp.port = x[0]
            comports.append(cp)
        print(comports)
        #---------------------------------------------    
        serialPort = 'COM3'#'/dev/ttyUSB2'#
        serialBaud = 115200

        # Connect to serial port
        print('Trying to connect to ' + str(serialPort) +
              ' at ' + str(serialBaud) + ' BAUD.')
        try:
            ser = serial.Serial(serialPort, serialBaud, timeout=4)
            print('Connected!')
        except:
            print("Failed to connect with " + str(serialPort) +
                  ' at ' + str(serialBaud) + ' BAUD.')
    #----------------------------------------------
        return ser


#Capture single trace
def capture_cw305(oscope,target,num_of_samples=600,isshort= False,channel='C2', board = "CW305", plain_text = [0],key=[0]):
    
    if (oscope.start_trigger()==False):
        print("Triggering Error!")
        return

    target.fpga_write(target.REG_CRYPT_KEY, key[::-1])# write key 
    target.fpga_write(target.REG_CRYPT_TEXTIN,plain_text[::-1])  # write plaintext
    target.usb_trigger_toggle() 
    

    # Get trace: if Lecroy has not stopped yet, discard this trace
    if(oscope.wait_for_trigger()==False):
        return None,[0]

    trc=oscope.get_channel(num_of_samples,isshort,channel)# Capture trace
    
    output = target.fpga_read(target.REG_CRYPT_CIPHEROUT, 16) # read ciphertext
    return trc, output
#Capture without sending the inputs
def capture_nopt(oscope,num_of_samples=600,isshort= False,channel='C2'):    
    if (oscope.start_trigger()==False):
        print("Triggering Error!")
        return

    # Get trace: if Lecroy has not stopped yet, discard this trace
    if(oscope.wait_for_trigger()==False):
      #i=i-1;
      #continue;
        return

    trc=oscope.get_channel(num_of_samples,isshort,channel)
    return trc

#Capture for Picochip board
# def capture_pico(oscope,ser,num_of_samples=120000,isshort= False,channel='C2',plain_text = [50]):
#     print('oscope_in_capture',oscope)
#     print('ser_in_capture',ser)
#     ser.write([82]) 
#     print('Send soft reset(R) to clear variables')
#     Ack = ser.read(1)
#     print('Acknowledgement(A) from DUT:',Ack)
   
#     print('plaintext',bytearray(plain_text))
    
#     if (oscope.start_trigger()==False):
#         print("Triggering Error!")
#         return
#     ser.write(plain_text)
#     print('send plaintext')

    

#     # Get trace: if Lecroy has not stopped yet, discard this trace
#     #for i in range(10):
#     if(oscope.wait_for_trigger()==False):
#         #i=i-1;
#         #continue;
#         return

#     trc=oscope.get_channel(num_of_samples,isshort,channel)
#     ciphertext = bytearray(ser.read(2))
#     print('AES sbox output',ciphertext)
    
    
   
#     return trc

In [ ]:
#%run Scope.ipynb
from tqdm.notebook import trange
from tqdm import tnrange
import random

#----------- libraries for serial(Dut Pico board) ---------
import serial.tools.list_ports
from serial import Serial
from serial.tools import list_ports
#dut_setup is used to setup the dut. for CW305 you can provide a bitfile as extra parameter
#tar = dut_setup(board="pico")#, bitfile ="D:/Riscure_Trace/Dillibabu/Ascon/cw305_top.bit")
  #-------------- Serial object -----------------
port_list = list(serial.tools.list_ports.comports())
print(port_list)
if len(port_list) == 0:
    print('No Available Serial Port')
else:
    for i in range(0,len(port_list)):
        print(port_list[i])

#----------------------------------------------
comports = []
for x in list_ports.comports():
    cp = Serial()
    cp.port = x[0]
    comports.append(cp)
print(comports)
#---------------------------------------------    
serialPort = 'COM3'#'/dev/ttyUSB2'#
serialBaud = 115200

# Connect to serial port
print('Trying to connect to ' + str(serialPort) +
      ' at ' + str(serialBaud) + ' BAUD.')
try:
    ser = serial.Serial(serialPort, serialBaud, timeout=4)
    print('Connected!')
except:
    print("Failed to connect with " + str(serialPort) +
          ' at ' + str(serialBaud) + ' BAUD.')

print('ser',ser)
#-------------------------scope ------------------------
# def scope_setup_1(num_of_samples=120000,sample_rate=50E6,isshort='False',vdiv=2.5E-3,trg_delay="0",trg_level="1.65V",voffset='0'):

#     # Acquisition setup: find this setting through the capturing traces from the setup program
#     #num_of_traces=10000                                          # number of traces
#     # num_of_samples=600                                      # number of samples
#     # sample_rate=10E9                                           # Sample Rate
#     # isshort=False                                             # Type of the samples: short or float
#     # vdiv=2.5E-3 #1.8E-2                                                 # Vertical resolution: V/div
#     # trg_delay ="0"# "-311.2US"                                      # Trigger delay: negative means post trigger (S)
#     # trg_level = "1.65V"                                            # Trigger level: start capture when trigger passes it
#     # #isenc = True                                                # Perform encryption/decryption
#     # voffset="0"#"-11.4 mV"                                          # Vertical Offset

#     # In most common cases, you do not have to change anything below this line
#     # Compute the setup parameters from above
#     xscale=1/sample_rate                                        # sampling interval (s)
#     duration=xscale*num_of_samples                              # sample duration (s)
#     # For short type of acquisition, the captured trace is scaled in order to store in a 16 bit integer
#     # yscale saves the scale value, in case you need to reconstruct the real traces
#     # For float type of acquisition, the samples are exactly the real samples on the trace, so yscale=1
#     if(isshort):
#         yscale=vdiv/(65536/10.0)
#         print("isshort")
#     else:
#         yscale=1
#         print("False")
#     timebase=str(xscale*num_of_samples/10)+"S"                  # timebase: s/div


#     #Open scope
#     oscope = Scope()
#     # setup the scope
#     oscope.setup(str(vdiv)+"V",timebase,str(sample_rate/1E6)+"MS/s",str(duration)+"S",voffset)
#     # set trigger
#     oscope.set_trigger(trg_delay,trg_level)
#     print('in scope object')
#     return oscope

#-----------------------------------------------------
#sco = scope_setup_1(isshort=True)
#for i in range(10):

    #traces = []
    #output=[]
plain_len=1
key= [43]#,126,21,22,40,174,210,166,171,247,21,136,9,207,79,60]
plain_text =[50]

print('ser',ser)

for i in trange(100):



    
    ser.write([82]) 
    print('Send soft reset(R) to clear variables')
    Ack = ser.read(1)
    print('Acknowledgement(A) from DUT:',Ack)
   
    print('plaintext',bytearray(plain_text))
    
#     if (oscope.start_trigger()==False):
#         print("Triggering Error!")
#         return
    ser.write(plain_text)
    print('send plaintext')

    

    # Get trace: if Lecroy has not stopped yet, discard this trace
#     #for i in range(10):
#     if(oscope.wait_for_trigger()==False):
#         #i=i-1;
#         #continue;
#         return

#     trc=oscope.get_channel(num_of_samples,isshort,channel)
    ciphertext = bytearray(ser.read(2))
    print('AES sbox output',ciphertext)
    print('i',i)
    
    
    
   
    #traces.append(trace)

    
#sco.close()
tar.dis()

In [ ]:
from math import sin, pi, ceil
from random import random
from typing import List

from spidersdk import Spider
from spidersdk.glitchpattern import PATTERN_STATE_LIMIT


def block(min_voltage: float, max_voltage: float, duration: float) -> List[float]:
    """
    Generate new pattern from the selected template

    :param min_voltage:  the minimum voltage level of the template function
    :param max_voltage: the maximum voltage level of the template function
    :param duration: the length of the pattern in seconds
    :return: the resulting pattern as a list of doubles. Every double represents a voltage level with
    a duration of 'Spider.MIN_SEC' seconds.
    """
    result = []

    number_of_segments = int(ceil(duration / Spider.MIN_SEC / 2))
    result.extend([min_voltage] * number_of_segments)
    result.extend([max_voltage] * number_of_segments)

    return result


def sine(min_voltage: float, max_voltage: float, duration: float) -> List[float]:
    """
    Generate new pattern from the selected template

    :param min_voltage:  the minimum voltage level of the template function
    :param max_voltage: the maximum voltage level of the template function
    :param duration: the length of the pattern in seconds
    :return: the resulting pattern as a list of doubles. Every double represents a voltage level with
    a duration of 'Spider.MIN_SEC' seconds.
    """
    result = []

    segment_length = duration / PATTERN_STATE_LIMIT
    number_of_segments = int(duration / Spider.MIN_SEC)
    if number_of_segments > PATTERN_STATE_LIMIT:
        number_of_segments = PATTERN_STATE_LIMIT
    voltage_offset = (max_voltage + min_voltage) / 2.0
    voltage_range = (max_voltage - min_voltage) / 2.0
    for k in range(0, number_of_segments):
        current_length = 0
        while current_length < segment_length:
            result.append(voltage_offset + (sin(2 * pi * (k / float(number_of_segments))) * voltage_range))
            current_length += Spider.MIN_SEC

    return result


def ramp(min_voltage: float, max_voltage: float, duration: float) -> List[float]:
    """
    Generate new pattern from the selected template

    :param min_voltage:  the minimum voltage level of the template function
    :param max_voltage: the maximum voltage level of the template function
    :param duration: the length of the pattern in seconds
    :return: the resulting pattern as a list of doubles. Every double represents a voltage level with
    a duration of 'Spider.MIN_SEC' seconds.
    """
    result = []

    segment_length = duration / PATTERN_STATE_LIMIT
    number_of_segments = int(duration / Spider.MIN_SEC)
    if number_of_segments > PATTERN_STATE_LIMIT:
        number_of_segments = PATTERN_STATE_LIMIT
    increment = (max_voltage - min_voltage) / number_of_segments
    for k in range(0, number_of_segments):
        current_length = 0
        while current_length < segment_length:
            result.append(min_voltage + (k * increment))
            current_length += Spider.MIN_SEC

    return result


def randomized(min_voltage: float, max_voltage: float, duration: float) -> List[float]:
    """
    Generate new pattern from the selected template

    :param min_voltage:  the minimum voltage level of the template function
    :param max_voltage: the maximum voltage level of the template function
    :param duration: the length of the pattern in seconds
    :return: the resulting pattern as a list of doubles. Every double represents a voltage level with
    a duration of 'Spider.MIN_SEC' seconds.
    """
    result = []

    segment_length = duration / PATTERN_STATE_LIMIT
    number_of_segments = int(duration / Spider.MIN_SEC)
    if number_of_segments > PATTERN_STATE_LIMIT:
        number_of_segments = PATTERN_STATE_LIMIT
    voltage_range = max_voltage - min_voltage
    for k in range(0, number_of_segments):
        current_length = 0
        value = (random() * voltage_range) + min_voltage
        while current_length < segment_length:
            result.append(value)
            current_length += Spider.MIN_SEC

    return result

In [ ]:
pip install D:\ChipWhisperer5_64\cw\home\portable\chipwhisperer\Dillibabu_expts\test_spider\python\dist\spidersdk-1.5.3-py3-none-any.whl

In [2]:
"""
 This script demonstrates how to use the Arbitrary Glitch Pattern Generator
"""

import serial

#from examples.templates import sine
from spidersdk import Spider
from spidersdk.glitchpattern import GlitchPattern, GlitchSegment
from spidersdk.glitchpatterngenerator import GlitchPatternGenerator
from tqdm.notebook import trange
from tqdm import tnrange
import random

#----------- libraries for serial(Dut Pico board) ---------
import serial.tools.list_ports
from serial import Serial
from serial.tools import list_ports

def from_templates():
    generator = generator_setup()

    ''' We will use the sine template for this example'''
    min_voltage = 2.3  # The lowest point of our sine wave will be 2.3V
    max_voltage = 3.3  # The highest point will be 3.3V
    duration = 1e-3  # The sine wave will have a period of 1ms
    sine_template = sine(min_voltage, max_voltage, duration)
    sine_pattern = GlitchPattern.of(sine_template)  # Create a Glitch Pattern from the sine template
    generator.set_glitch_pattern(sine_pattern)  # Send the glitch pattern to the Spider

    ''' Now we start the generator. It will wait for a trigger on the specified pin.'''
    ''' If a trigger is detected, it will wait for the specified delay time and'''
    ''' then generate the specified pattern'''
    generator.start()


def from_list():
    generator = generator_setup()

    ''' Every entry in the list is assumed to have a duration of 8ns'''
    ''' To hold a voltage level for longer, simply duplicate the entry'''
    voltages = [3.2, 3.2,  # 3.2V for 16 ns
                3.0, 3.0,  # 3.0V for 16 ns
                2.8, 2.8, 2.8, 2.8,  # 2.8V for 32 ns
                3.0, 3.0,  # 3.0V for 16 ns
                3.2, 3.2]  # 3.2V for 16 ns
    glitch_pattern = GlitchPattern.of(voltages)
    generator.set_glitch_pattern(glitch_pattern)

    ''' Now we start the generator. It will wait for a trigger on the specified pin.'''
    ''' If a trigger is detected, it will wait for the specified delay time and'''
    ''' then generate the specified pattern'''
    generator.start()


def manual():
    generator = generator_setup()

    ''' This is done by adding segments with a voltage level and a duration'''
    glitch_pattern = GlitchPattern()
    glitch_pattern.add(GlitchSegment(3.2, 16e-9))  # 3.2V for 16ns
    glitch_pattern.add(GlitchSegment(3.0, 16e-9))  # 3.0V for 16ns
    glitch_pattern.add(GlitchSegment(2.5, 32e-9))  # 2.5V for 32ns
    glitch_pattern.add(GlitchSegment(3.0, 16e-9))  # 3.0V for 16ns
    glitch_pattern.add(GlitchSegment(3.2, 16e-9))  # 3.2V for 16ns
    generator.set_glitch_pattern(glitch_pattern)

    ''' Now we start the generator. It will wait for a trigger on the specified pin.'''
    ''' If a trigger is detected, it will wait for the specified delay time and'''
    ''' then generate the specified pattern'''
    generator.start()


def generator_setup() -> GlitchPatternGenerator:
    spider_core1 = Spider(Spider.CORE1, com_port)  # Instantiate a Spider object using CORE1
    spider_core1.reset_settings()  # Reset all the existing settings of CORE1
    generator = GlitchPatternGenerator(spider_core1)  # Initialize the glitch pattern generator on CORE1

    ''' General setup of the pattern generator'''
    generator.normal_voltage = 3.3  # 3.3V normal supply voltage
    generator.glitch_out = Spider.GLITCH_OUT1  # Generate pattern on Glitch Out 1
    generator.glitch_delay = 0.1  # Glitch delay is 100 ms

    ''' Normally we would set up the trigger here, but we don't have a trigger generator.
        In this case, we can just ignore it. This will cause the pattern generator to start as soon as
        we call generator.start'''
    trigger_pin = 0                                                 # Set the trigger pin to CORE1.GPIO_Pin0
    trigger_edge = Spider.RISING_EDGE                               # Set the trigger edge to rising
    generator.set_trigger(trigger_pin, trigger_edge)                # Enable the trigger
    print("set up glitches after trigger event")

    return generator


if __name__ == '__main__':
    com_port = serial.Serial()
    com_port.port = "COM6"  # FIXME: need an actual COM port number
    com_port.open()
    
     #-------------- Serial object -----------------
    port_list = list(serial.tools.list_ports.comports())
    print(port_list)
    if len(port_list) == 0:
        print('No Available Serial Port')
    else:
        for i in range(0,len(port_list)):
            print(port_list[i])

    #----------------------------------------------
    comports = []
    for x in list_ports.comports():
        cp = Serial()
        cp.port = x[0]
        comports.append(cp)
    print(comports)
    #---------------------------------------------    
    serialPort = 'COM3'#'/dev/ttyUSB2'#
    serialBaud = 115200

    # Connect to serial port
    print('Trying to connect to ' + str(serialPort) +
          ' at ' + str(serialBaud) + ' BAUD.')
    try:
        ser = serial.Serial(serialPort, serialBaud, timeout=4)
        print('Connected!')
    except:
        print("Failed to connect with " + str(serialPort) +
              ' at ' + str(serialBaud) + ' BAUD.')

    print('ser',ser)
    plain_text = [50]

    ''' There are 3 ways to create a pattern'''
    ser.write([82]) 
    print('Send soft reset(R) to clear variables')
    Ack = ser.read(1)
    print('Acknowledgement(A) from DUT:',Ack)
   
    print('plaintext',bytearray(plain_text))
    #-------------- pattern ------------------------------------
#     from_templates()
#     #------------- voltage -------------------------------------
#     from_list()
#     #------------- manual --------------------------------------
#     manual()
    #-----------------------------------------------------------
    
#     if (oscope.start_trigger()==False):
#         print("Triggering Error!")
#         return
    ser.write(plain_text)
    print('send plaintext')

    

    # Get trace: if Lecroy has not stopped yet, discard this trace
#     #for i in range(10):
#     if(oscope.wait_for_trigger()==False):
#         #i=i-1;
#         #continue;
#         return

#     trc=oscope.get_channel(num_of_samples,isshort,channel)
    ciphertext = bytearray(ser.read(2))
    print(ciphertext)
    ''' 0: Use a pre-defined pattern'''
    #from_templates()
    ''' 1: Use a list of voltages'''
    #from_list()
    ''' 2: Manually create a glitch pattern'''
    #manual()

    com_port.close()


[<serial.tools.list_ports_common.ListPortInfo object at 0x000001C4B823EB30>, <serial.tools.list_ports_common.ListPortInfo object at 0x000001C4B823EB90>, <serial.tools.list_ports_common.ListPortInfo object at 0x000001C4B823EB60>, <serial.tools.list_ports_common.ListPortInfo object at 0x000001C4B823EBC0>]
COM3 - USB Serial Port (COM3)
COM6 - Test Tool 1.x (COM6)
COM1 - Communications Port (COM1)
COM7 - TRINAMIC Stepper Device (COM7)
[Serial<id=0x1c4b823d960, open=False>(port='COM3', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False), Serial<id=0x1c4b823d990, open=False>(port='COM6', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False), Serial<id=0x1c4b823dab0, open=False>(port='COM1', baudrate=9600, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False), Serial<id=0x1c4b823d8a0, open=False>(port='COM7', baudrate=9600, bytesize=8, parity='N'

In [ ]:
import serial
from spidersdk import Glitcher, Spider

import serial.tools.list_ports
from serial import Serial
from serial.tools import list_ports

def example_glitcher():
    #com_port = serial.Serial()
    #com_port.port = "COM6"  # FIXME: need an actual COM port number
    #com_port.open()
    
     #-------------- Serial object -----------------
    port_list = list(serial.tools.list_ports.comports())
    print(port_list)
    if len(port_list) == 0:
        print('No Available Serial Port')
    else:
        for i in range(0,len(port_list)):
            print(port_list[i])

    #----------------------------------------------
    comports = []
    for x in list_ports.comports():
        cp = Serial()
        cp.port = x[0]
        comports.append(cp)
    print(comports)
    #---------------------------------------------    
    serialPort = 'COM6'#'/dev/ttyUSB2'#
    serialBaud = 115200

    # Connect to serial port
    print('Trying to connect to ' + str(serialPort) +
          ' at ' + str(serialBaud) + ' BAUD.')
    try:
        ser_6 = serial.Serial(serialPort, serialBaud, timeout=4)
        print('Connected!')
    except:
        print("Failed to connect with " + str(serialPort) +
              ' at ' + str(serialBaud) + ' BAUD.')

    print('ser_6',ser_6)
    #com_port = 'COM6'
    #com_port.open()
    
    spider_core1 = Spider(Spider.CORE1, ser_6)  # Instantiate a Spider object using CORE1
    spider_core1.reset_settings()  # Clear all previous settings in CORE1
    glitcher = Glitcher(spider_core1, 0)  # Instantiate a Glitcher machine in Spider CORE1
    # Trigger Input @ Pin 0
    glitcher.clear_glitches()  # Clear exiting glitches
    glitcher.set_normal_vcc(Spider.GLITCH_OUT1, 3.3)  # Set normal Vcc of glitch out 1 to 3.3 Volts, this value will be effective until next set_normal_vcc() call
    #glitcher.set_normal_vcc(Spider.GLITCH_OUT2, 1.8)  # Set normal Vcc of glitch out 2 to 1.8 Volts, this value will be effective until next set_normal_vcc() call
    glitcher.set_glitch_vcc(Spider.GLITCH_OUT1, 0.0)  # Set glitch Vcc of glitch out 1 to 0.0 Volts, this value will be effective until next set_glitch_vcc() call
    #glitcher.set_glitch_vcc(Spider.GLITCH_OUT2, 0.5)  # Set glitch Vcc of glitch out 2 to 0.0 Volts, this value will be effective until next set_normal_vcc() call
    glitcher.add_glitch(Spider.GLITCH_OUT1, 0, 4e-9)  # Assign the 1st glitch to glitch out 1:
    # Wait time = 0.0ns
    # Glitch time = 4.0ns
    #glitcher.add_glitch(Spider.GLITCH_OUT2, 48e-9, 12e-9)  # Assign the 2nd glitch to glitch out 2:
    # Wait time = 48.0 ns after 1st glitch
    # Glitch time = 100.0 ns
    #glitcher.add_glitch(Spider.GLITCH_OUT_ALL, 100e-9, 100e-9)  # Assign the 3rd glitch to both glitch out 1 and 2:
    # Wait time = 100.0 ns after the 2nd glitch
    # Glitch time = 100.0 ns
    glitcher.set_power(Spider.VOLTAGE_OUT1, 1.0)  # Set Power to 100% using Voltage out 1
    #glitcher.set_power(Spider.VOLTAGE_OUT2, 0.5)  # Set Power to 50% using Voltage out 2
    glitcher.arm(edge=Spider.RISING_EDGE, trigger_count=1)  # Arm the glitcher to detect trigger
    # Count 1 rising edge and start glitching
    # Trigger event then can be arranged after arm() call
    # An example of generating a trigger signal using CORE2 is given below:
    # Connect Core2.Pin0 to Glitcher class trigger in pin
    # Connect Core2.Pin31 can be optionally connected to Scope
    #spider_core2 = Spider(Spider.CORE2, com_port)
    #spider_core2.reset_settings()
    #spider_core2.clr_bits(0, [0, 31])
    #spider_core2.set_bits(0, [0, 31])
    print(glitcher.is_glitcher_done())  # Check if the glitcher is done. User may implement a timeout here.
    print("Glitch done")
    ser_6.close()


if __name__ == '__main__':
    #----------example_glitcher()
     #-------------- Serial object -----------------
    port_list = list(serial.tools.list_ports.comports())
    print(port_list)
    if len(port_list) == 0:
        print('No Available Serial Port')
    else:
        for i in range(0,len(port_list)):
            print(port_list[i])

    #----------------------------------------------
    comports = []
    for x in list_ports.comports():
        cp = Serial()
        cp.port = x[0]
        comports.append(cp)
    print(comports)
    #---------------------------------------------    
    serialPort = 'COM3'#'/dev/ttyUSB2'#
    serialBaud = 115200

    # Connect to serial port
    print('Trying to connect to ' + str(serialPort) +
          ' at ' + str(serialBaud) + ' BAUD.')
    try:
        ser = serial.Serial(serialPort, serialBaud, timeout=4)
        print('Connected!')
    except:
        print("Failed to connect with " + str(serialPort) +
              ' at ' + str(serialBaud) + ' BAUD.')

    print('ser',ser)
    plain_text = [50]

    ''' There are 3 ways to create a pattern'''
    ser.write([82]) 
    print('Send soft reset(R) to clear variables')
    Ack = ser.read(1)
    print('Acknowledgement(A) from DUT:',Ack)
   
    print('plaintext',bytearray(plain_text))
    
    
#     if (oscope.start_trigger()==False):
#         print("Triggering Error!")
#         return
    ser.write(plain_text)
    print('send plaintext')
    #------------------------------ Glitcher -------------------------
    example_glitcher()
    #-----------------------------------------------------------------
    

    # Get trace: if Lecroy has not stopped yet, discard this trace
#     #for i in range(10):
#     if(oscope.wait_for_trigger()==False):
#         #i=i-1;
#         #continue;
#         return

#     trc=oscope.get_channel(num_of_samples,isshort,channel)
    ciphertext = bytearray(ser.read(2))
    print(ciphertext)
    ''' 0: Use a pre-defined pattern'''
    #from_templates()
    ''' 1: Use a list of voltages'''
    #from_list()
    ''' 2: Manually create a glitch pattern'''
    #manual()

    ser.close()


In [3]:
  ser.close()